x# Entity Resolution using Deep Learning

In this notebook we're going to try to reproduce the results of some of the
systems described in the most complete survey [@barlaugsurvey2021] on entity
matching using deep learning techniques.
Additionally, where we can, we will compare the results obtained using those
techniques with logistic regression.

First things first: a few imports of modules and data.

In [1]:
import itertools
from dataclasses import dataclass

from matchescu.matching.entity_reference import RawComparison
!test -f ~/requirements.txt && pip install -r ~/requirements.txt

In [2]:
import os
import polars as pl

from matchescu.matching.ml.datasets import CsvDataSource

Just like we've done previously, we'll be using only the Abt-Buy dataset.

In [3]:
LANG="en"
DATADIR = os.path.abspath("../../data")
LEFT_CSV_PATH = os.path.join(DATADIR, "abt-buy", "Abt.csv")
RIGHT_CSV_PATH = os.path.join(DATADIR, "abt-buy", "Buy.csv")
GROUND_TRUTH_PATH = os.path.join(DATADIR, "abt-buy", "abt_buy_perfectMapping.csv")

Unlike with our previous approaches, we're not quite ready to construct a
feature matrix.
While we could definitely use the previous extraction traits to provide a
feature matrix containing the similarities of co-referent attributes, we want to
improve upon our work so far.
To do so, we're going to attempt to implement Deepmatcher from scratch as it is
described in the [@deepmatcher2018] paper.
An important note is that we'll be implementing the Hybrid Deepmatcher approach
because it is the one with the highest success.

Firstly, we must embed attributes into sequences of word vectors.
To do this we're going to use NLTK to tokenize the input and fasttext to create
the character embeddings for every word.

In [4]:
import fasttext
import nltk

from fasttext.util import download_model


nltk.download("punkt")
download_model(LANG, if_exists="ignore")
ft_model = fasttext.load_model(f"cc.{LANG}.300.bin")

[nltk_data] Downloading package punkt to /Users/cusi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Now that we have downloaded the `punkt` package locally, we can easily tokenize
the words we find in each attribute in our two data sources.
Let's write a function that does that.

In [23]:
# Model hyperparameters
SIF_ALPHA = 1.0
INPUT_SIZE = ft_model.get_dimension()
N_EPOCHS = 2

In [6]:
from typing import Any, Generator, Iterable

import torch
from matchescu.typing import DataSource, Record


@dataclass
class TokenEmbedding:
    token: str
    embedding: torch.Tensor


def tokenize_attribute_value(value: Any) -> Generator[str, None, None]:
    if value is None:
        return
    if not isinstance(value, str):
        value = str(value)
    value = value.lower()
    yield from nltk.word_tokenize(value)


def embed_str(value: str) -> torch.Tensor:
    return torch.from_numpy(ft_model.get_word_vector(value))


def token_embedding(value: str) -> TokenEmbedding:
    return TokenEmbedding(value, embed_str(value))


def tokenize_words(record: tuple) -> list[list[TokenEmbedding]]:
    return [
        [
            token_embedding(token)
            for token in tokenize_attribute_value(value)
        ]
        for value in record
    ]


def tokenize_data_source(ds: DataSource[Record]) -> Generator[list[list[TokenEmbedding]], None, None]:
    yield from (tokenize_words(record) for record in ds)


def build_normalized_unigram_frequencies(processed_data_sources: Iterable[list[list[TokenEmbedding]]]) -> dict[str, float]:
    token_frequencies = {}
    word_count = 0
    for ds in processed_data_sources:
        for tokenized_record in ds:
            for token_embedding in tokenized_record:
                token_frequency = token_frequencies.get(token_embedding.token, 0)
                token_frequency += 1
                token_frequencies[token_embedding.token] = token_frequency
                word_count += 1
    return {
        token: token_frequency / word_count
        for token, token_frequency in token_frequencies.items()
    }


def compute_token_weights(token_embeddings: list[TokenEmbedding], frequency_table: dict[str, float], a: float = 1.0) -> torch.Tensor:
    return torch.Tensor([
        (a / (a + frequency_table[te.token]))
        for te in token_embeddings
    ])

abt = CsvDataSource(name="abt", traits=[]).read_csv(LEFT_CSV_PATH)
buy = CsvDataSource(name="buy", traits=[]).read_csv(RIGHT_CSV_PATH)
tokenized_abt = list(tokenize_data_source(abt))
tokenized_buy = list(tokenize_data_source(buy))
token_frequency_table = build_normalized_unigram_frequencies(itertools.chain(tokenized_abt, tokenized_buy))

In [7]:
class SifTensorTrait:
    def __init__(self, frequency_table: dict[str, float], alpha: float = 1.0, input_dim: int = 300) -> None:
        self._ft = frequency_table
        self._a = alpha
        self._n = input_dim

    def _extract_attribute_tensors(self, record: Record) -> Generator[torch.Tensor, None, None]:
        for attr_value in record:
            token_embeddings = list(map(token_embedding, tokenize_attribute_value(attr_value)))
            if len(token_embeddings) < 1:
                yield torch.zeros(self._n)
                continue
            token_weights = compute_token_weights(
                token_embeddings, self._ft, self._a
            ).reshape(len(token_embeddings), 1)
            word_embeddings = torch.atleast_2d(
                torch.stack([te.embedding for te in token_embeddings])
            )
            weighted_embeddings = token_weights * word_embeddings
            colwise_weighted_sum = weighted_embeddings.sum(dim=0)
            total_weight = token_weights.sum().float()
            yield colwise_weighted_sum / total_weight

    def __call__(self, record: Record) -> tuple[torch.Tensor, ...]:
        return tuple(t for t in self._extract_attribute_tensors(record))


In [8]:
sif = SifTensorTrait(token_frequency_table, SIF_ALPHA, INPUT_SIZE)
abt = CsvDataSource(name="abt", traits=[sif]).read_csv(LEFT_CSV_PATH)
buy = CsvDataSource(name="buy", traits=[sif]).read_csv(RIGHT_CSV_PATH)
gt = set(
    pl.read_csv(
        os.path.join(DATADIR, "abt-buy", "abt_buy_perfectMapping.csv"),
        ignore_errors=True,
    ).iter_rows()
)

In [9]:
from matchescu.matching.ml.datasets import RecordLinkageDataSet
cmp_config = (
    RawComparison()
    .euclidian_distance("name", 1, 1)
    .euclidian_distance("description", 2, 2)
    .euclidian_distance("name_manufacturer", 1, 3)
    .euclidian_distance("description_manufacturer", 2, 3)
    .euclidian_distance("price", 3, 4)
)

ds = RecordLinkageDataSet(abt, buy, gt).attr_compare(cmp_config).cross_sources()
display(ds.feature_matrix)

name,description,name_manufacturer,description_manufacturer,price
f64,f64,f64,f64,f64
0.632966,0.759937,1.290251,1.167136,0.000017
0.940538,1.846598,1.290251,1.167136,0.000017
0.624203,0.551165,1.247905,1.117197,0.000017
0.735849,1.766785,1.36398,1.279892,0.000017
0.942468,0.588245,1.247905,1.117197,0.000017
…,…,…,…,…
0.692637,0.844414,1.831503,1.760053,2.695225
1.1028,0.530939,0.924538,0.872439,2.695225
0.996801,0.530939,1.342565,1.245386,2.695225


Let's add a few more helper functions that enable us to train an ANN classifier.

In [28]:
import torch
from torch import nn
from torch.nn.modules import loss
from torch.optim import Adam, Optimizer
from torch.utils.data import Dataset, DataLoader

def get_torch_device():
    return f"mps:{torch.mps.device_count()-1}" if torch.mps.is_available() else "cpu"


def get_torch_generator():
    return torch.Generator(device=get_torch_device())


def create_dataloader(input_dataset: Dataset, batch_size: int = 32, shuffle: bool = True):
    return DataLoader(
        input_dataset,
        batch_size=batch_size,
        shuffle=shuffle,
    )


def random_split(input_dataset: Dataset, train_proportion: float = 0.6, cv_proportion = 0.2) -> tuple[Dataset, Dataset, Dataset]:
    assert train_proportion + cv_proportion < 0.9

    train, cv, test = torch.utils.data.random_split(
        input_dataset,
        [train_proportion, cv_proportion, 1.0 - train_proportion - cv_proportion],
        generator=torch.Generator().manual_seed(42)
    )
    return train, cv, test


def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    x_validation_loader: DataLoader,
    loss_function: loss._Loss,
    optimizer: Optimizer,
    epochs: int = 10
):
    dev = get_torch_device()
    model.to(dev)
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for X, y in train_loader:
            X = X.to(dev)
            y = y.to(dev)
            optimizer.zero_grad()
            out = model(X)
            loss_value = loss_function(out, y.unsqueeze(1))
            loss_value.backward()
            optimizer.step()
            train_loss += loss_value.item()
        train_loss /= len(train_loader)

        # Validation phase
        model.eval()
        x_validation_loss = 0.0
        with torch.no_grad():
            for X, y in x_validation_loader:
                X = X.to(dev)
                y = y.to(dev)
                out = model(X)
                loss_value = loss_function(out, y.unsqueeze(1))
                x_validation_loss += loss_value.item()
        x_validation_loss /= len(x_validation_loader)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {x_validation_loss:.4f}")
    return model

In [ ]:
from matchescu.matching.ml.modules import HighwayMatchClassifier


dataset = ds.create_torch_dataset()
matcher = HighwayMatchClassifier(5)
train, cv, test = random_split(dataset)

mse_loss = loss.MSELoss()
train_model(matcher, create_dataloader(train), create_dataloader(cv), mse_loss, Adam(matcher.parameters(), lr=0.01), N_EPOCHS)

In [11]:
matcher.eval()
test_loader = create_dataloader(test)
test_loss = 0.0
with torch.no_grad():
    for features, targets in test_loader:
        outputs = matcher(features)
        loss = mse_loss(outputs, targets.unsqueeze(1))
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")


SyntaxError: expected ':' (2294033000.py, line 13)